In [1]:
import pandas as pd

In [2]:
pref = pd.read_csv("COVID-19-jp/nhk_news_covid19_prefectures_daily_data.csv")
pref["日付"] = pd.to_datetime(pref["日付"] )
print(len(pref))
pref.tail()

24816


,日付,都道府県コード,都道府県名,各地の感染者数_1日ごとの発表数,各地の感染者数_累計,各地の死者数_1日ごとの発表数,各地の死者数_累計
24811,2021-06-22,47,沖縄県,98,20232,4,171
24812,2021-06-23,47,沖縄県,96,20328,0,171
24813,2021-06-24,47,沖縄県,62,20390,4,175
24814,2021-06-25,47,沖縄県,82,20472,4,179
24815,2021-06-26,47,沖縄県,74,20546,0,179


In [3]:
latest_date = pref["日付"].max().strftime("%Y-%m-%d")

In [4]:
all_jp = pref.groupby(["都道府県名"], as_index=False).sum()
del all_jp["各地の感染者数_累計"]
del all_jp["各地の死者数_累計"]
#all_jp = all_jp.rename(columns={"各地の感染者数合計":"各地の感染者数_1日ごとの発表数"})
all_jp = all_jp.rename(columns={"各地の感染者数_1日ごとの発表数":"各地の感染者数合計"})
all_jp = all_jp.rename(columns={"各地の死者数_1日ごとの発表数":"各地の死者数合計"})
all_jp.tail()

,都道府県名,都道府県コード,各地の感染者数合計,各地の死者数合計
42,静岡県,11616,9217,150
43,香川県,19536,2090,30
44,高知県,20592,1788,24
45,鳥取県,16368,466,2
46,鹿児島県,24288,3653,38


## 都道府県ごとの最新データを付与

In [5]:
# 直近１週間の感染者数
pref["直近1週間の感染者数"] = pref["各地の感染者数_1日ごとの発表数"].rolling(7).sum()
pref.tail(7)

,日付,都道府県コード,都道府県名,各地の感染者数_1日ごとの発表数,各地の感染者数_累計,各地の死者数_1日ごとの発表数,各地の死者数_累計,直近1週間の感染者数
24809,2021-06-20,47,沖縄県,61,20101,0,167,617.0
24810,2021-06-21,47,沖縄県,33,20134,0,167,596.0
24811,2021-06-22,47,沖縄県,98,20232,4,171,587.0
24812,2021-06-23,47,沖縄県,96,20328,0,171,568.0
24813,2021-06-24,47,沖縄県,62,20390,4,175,533.0
24814,2021-06-25,47,沖縄県,82,20472,4,179,529.0
24815,2021-06-26,47,沖縄県,74,20546,0,179,506.0


### 感染者数前日比

In [6]:
import numpy as np
pref["新規感染者数前日比"] = pref["各地の感染者数_1日ごとの発表数"].pct_change()
# 一番古い日付だけNANを設定
pref.loc[pref["日付"]==pref["日付"].min(),"新規感染者数前日比"] = np.nan
pref["直近1週間の新規感染者数前日比平均"] = pref["新規感染者数前日比"].rolling(7).mean()
pref["新規感染者数先週比"] = pref["各地の感染者数_1日ごとの発表数"].pct_change(periods=7)

pref.tail(10)

,日付,都道府県コード,都道府県名,各地の感染者数_1日ごとの発表数,各地の感染者数_累計,各地の死者数_1日ごとの発表数,各地の死者数_累計,直近1週間の感染者数,新規感染者数前日比,直近1週間の新規感染者数前日比平均,新規感染者数先週比
24806,2021-06-17,47,沖縄県,97,19857,4,167,779.0,-0.156522,0.005376,-0.415663
24807,2021-06-18,47,沖縄県,86,19943,0,167,720.0,-0.113402,0.007248,-0.406897
24808,2021-06-19,47,沖縄県,97,20040,0,167,660.0,0.127907,0.013697,-0.382166
24809,2021-06-20,47,沖縄県,61,20101,0,167,617.0,-0.371134,0.008904,-0.413462
24810,2021-06-21,47,沖縄県,33,20134,0,167,596.0,-0.459016,0.012012,-0.388889
24811,2021-06-22,47,沖縄県,98,20232,4,171,587.0,1.969697,0.153185,-0.084112
24812,2021-06-23,47,沖縄県,96,20328,0,171,568.0,-0.020408,0.139589,-0.165217
24813,2021-06-24,47,沖縄県,62,20390,4,175,533.0,-0.354167,0.111354,-0.360825
24814,2021-06-25,47,沖縄県,82,20472,4,179,529.0,0.322581,0.173637,-0.046512
24815,2021-06-26,47,沖縄県,74,20546,0,179,506.0,-0.097561,0.141427,-0.237113


In [7]:
latest = pref.loc[pref["日付"]==pref["日付"].max()][["日付","都道府県名","各地の感染者数_1日ごとの発表数","各地の死者数_1日ごとの発表数","直近1週間の感染者数","新規感染者数前日比","直近1週間の新規感染者数前日比平均","新規感染者数先週比"]]
latest.head()

,日付,都道府県名,各地の感染者数_1日ごとの発表数,各地の死者数_1日ごとの発表数,直近1週間の感染者数,新規感染者数前日比,直近1週間の新規感染者数前日比平均,新規感染者数先週比
527,2021-06-26,北海道,47,5,273.0,0.424242,0.017342,-0.356164
1055,2021-06-26,青森県,11,0,23.0,4.500000,NaN,10.000000
1583,2021-06-26,岩手県,2,0,27.0,-0.500000,NaN,-0.714286
2111,2021-06-26,宮城県,5,1,40.0,-0.375000,0.307653,0.666667
2639,2021-06-26,秋田県,21,0,60.0,0.750000,0.986848,20.000000


In [8]:
all_jp = pd.merge(all_jp, latest, on="都道府県名")
all_jp.head()

,都道府県名,都道府県コード,各地の感染者数合計,各地の死者数合計,日付,各地の感染者数_1日ごとの発表数,各地の死者数_1日ごとの発表数,直近1週間の感染者数,新規感染者数前日比,直近1週間の新規感染者数前日比平均,新規感染者数先週比
0,三重県,12672,5243,110,2021-06-26,5,0,62.0,-0.772727,0.746204,0.250000
1,京都府,13728,16539,241,2021-06-26,20,0,90.0,1.000000,1.612670,0.250000
2,佐賀県,21648,2554,24,2021-06-26,1,0,9.0,inf,NaN,-0.666667
3,兵庫県,14784,40850,1282,2021-06-26,34,3,152.0,0.700000,0.173294,0.888889
4,北海道,528,41216,1382,2021-06-26,47,5,273.0,0.424242,0.017342,-0.356164


## 人口データを付与

In [9]:
pop = pd.read_excel("COVID-19-jp/2001stjin.xls",header=3)[["Unnamed: 1","計"]]
pop.rename(columns={"Unnamed: 1":"都道府県名"},inplace=True)
pop.rename(columns={"計":"人口"},inplace=True)
pop = pop.loc[pop["都道府県名"]!="合計"].reset_index(drop=True)

In [10]:
all_jp = pd.merge(all_jp, pop, on="都道府県名", how="left")
all_jp.head()

,都道府県名,都道府県コード,各地の感染者数合計,各地の死者数合計,日付,各地の感染者数_1日ごとの発表数,各地の死者数_1日ごとの発表数,直近1週間の感染者数,新規感染者数前日比,直近1週間の新規感染者数前日比平均,新規感染者数先週比,人口
0,三重県,12672,5243,110,2021-06-26,5,0,62.0,-0.772727,0.746204,0.250000,1813859
1,京都府,13728,16539,241,2021-06-26,20,0,90.0,1.000000,1.612670,0.250000,2545899
2,佐賀県,21648,2554,24,2021-06-26,1,0,9.0,inf,NaN,-0.666667,823810
3,兵庫県,14784,40850,1282,2021-06-26,34,3,152.0,0.700000,0.173294,0.888889,5549568
4,北海道,528,41216,1382,2021-06-26,47,5,273.0,0.424242,0.017342,-0.356164,5267762


## 人口当たりの感染者数

In [11]:
all_jp["１万人当たりの感染者数"] = all_jp["各地の感染者数合計"] / (all_jp["人口"] / 10000)
all_jp["10万人当たりの感染者数"] = all_jp["各地の感染者数合計"] / (all_jp["人口"] / 100000)
all_jp["直近1週間の10万人当たりの感染者数"] = all_jp["直近1週間の感染者数"] / (all_jp["人口"] / 100000)
all_jp.sort_values("都道府県コード")

,都道府県名,都道府県コード,各地の感染者数合計,各地の死者数合計,日付,各地の感染者数_1日ごとの発表数,各地の死者数_1日ごとの発表数,直近1週間の感染者数,新規感染者数前日比,直近1週間の新規感染者数前日比平均,新規感染者数先週比,人口,１万人当たりの感染者数,10万人当たりの感染者数,直近1週間の10万人当たりの感染者数
4,北海道,528,41216,1382,2021-06-26,47,5,273.0,0.424242,0.017342,-0.356164,5267762,78.241956,782.419555,5.182466
41,青森県,1056,2476,31,2021-06-26,11,0,23.0,4.500000,NaN,10.000000,1275783,19.407689,194.076892,1.802814
19,岩手県,1584,1659,46,2021-06-26,2,0,27.0,-0.500000,NaN,-0.714286,1235517,13.427577,134.275773,2.185320
11,宮城県,2112,9104,89,2021-06-26,5,1,40.0,-0.375000,0.307653,0.666667,2292385,39.714097,397.140969,1.744908
36,秋田県,2640,825,20,2021-06-26,21,0,60.0,0.750000,0.986848,20.000000,985416,8.372099,83.720987,6.088799
15,山形県,3168,2020,47,2021-06-26,0,0,0.0,NaN,NaN,-1.000000,1082296,18.664025,186.640254,0.000000
35,福島県,3696,4840,161,2021-06-26,17,0,76.0,0.888889,0.176118,0.700000,1881981,25.717582,257.175816,4.038298
38,茨城県,4224,10440,162,2021-06-26,31,0,186.0,-0.446429,0.369879,-0.060606,2921436,35.735850,357.358505,6.366732
27,栃木県,4752,6939,81,2021-06-26,37,0,163.0,0.370370,0.196981,1.642857,1965516,35.303707,353.037065,8.292988
37,群馬県,5280,8022,152,2021-06-26,5,0,23.0,0.250000,NaN,0.666667,1969439,40.732412,407.324116,1.167845


## 感染者数

### 平均値、中央値

In [12]:
all_jp["各地の感染者数合計"].describe()

count        47.000000
mean      16830.893617
std       30918.622555
min         466.000000
25%        2515.000000
50%        5023.000000
75%       10954.000000
max      172041.000000
Name: 各地の感染者数合計, dtype: float64

### TOP 10

In [13]:
all_jp.sort_values("各地の感染者数合計", ascending=False).reset_index(drop=True).head(10)

,都道府県名,都道府県コード,各地の感染者数合計,各地の死者数合計,日付,各地の感染者数_1日ごとの発表数,各地の死者数_1日ごとの発表数,直近1週間の感染者数,新規感染者数前日比,直近1週間の新規感染者数前日比平均,新規感染者数先週比,人口,１万人当たりの感染者数,10万人当たりの感染者数,直近1週間の10万人当たりの感染者数
0,東京都,6864,172041,2222,2021-06-26,534,4,3332.0,-0.049822,0.102846,0.376289,13834925,124.352680,1243.526799,24.083976
1,大阪府,14256,103038,2647,2021-06-26,88,7,704.0,-0.266667,0.108977,-0.207207,8849635,116.431921,1164.319206,7.955130
2,神奈川県,7392,66605,944,2021-06-26,231,0,1305.0,0.045249,0.045773,0.276243,9209442,72.322514,723.225142,14.170240
3,愛知県,12144,50930,955,2021-06-26,46,5,501.0,-0.303030,0.081240,-0.483146,7575530,67.229620,672.296196,6.613399
4,埼玉県,5808,46097,830,2021-06-26,96,0,641.0,-0.040000,0.066528,0.215190,7390054,62.377081,623.770814,8.673820
5,北海道,528,41216,1382,2021-06-26,47,5,273.0,0.424242,0.017342,-0.356164,5267762,78.241956,782.419555,5.182466
6,兵庫県,14784,40850,1282,2021-06-26,34,3,152.0,0.700000,0.173294,0.888889,5549568,73.609333,736.093332,2.738952
7,千葉県,6336,39811,704,2021-06-26,108,1,756.0,-0.142857,0.021467,-0.092437,6319772,62.994361,629.943612,11.962457
8,福岡県,21120,35414,519,2021-06-26,40,1,234.0,0.081081,0.068368,-0.148936,5129841,69.035278,690.352781,4.561545
9,沖縄県,24816,20546,179,2021-06-26,74,0,506.0,-0.097561,0.141427,-0.237113,1481547,138.679367,1386.793669,34.153490


## １万人あたりの感染者数

### 平均値、中央値

In [14]:
all_jp["１万人当たりの感染者数"].describe()

count     47.000000
mean      40.595893
std       29.470508
min        8.125725
25%       22.349224
50%       30.324114
75%       52.782350
max      138.679367
Name: １万人当たりの感染者数, dtype: float64

## 直近１週間の１０万人あたりの感染者数

In [15]:
all_jp.sort_values(["直近1週間の10万人当たりの感染者数"],ascending=False).head(10)

,都道府県名,都道府県コード,各地の感染者数合計,各地の死者数合計,日付,各地の感染者数_1日ごとの発表数,各地の死者数_1日ごとの発表数,直近1週間の感染者数,新規感染者数前日比,直近1週間の新規感染者数前日比平均,新規感染者数先週比,人口,１万人当たりの感染者数,10万人当たりの感染者数,直近1週間の10万人当たりの感染者数
28,沖縄県,24816,20546,179,2021-06-26,74,0,506.0,-0.097561,0.141427,-0.237113,1481547,138.679367,1386.793669,34.153490
26,東京都,6864,172041,2222,2021-06-26,534,4,3332.0,-0.049822,0.102846,0.376289,13834925,124.352680,1243.526799,24.083976
33,福井県,9504,1240,34,2021-06-26,18,0,144.0,0.058824,0.245500,1.250000,780053,15.896356,158.963558,18.460284
16,山梨県,10032,2085,21,2021-06-26,12,0,123.0,-0.142857,-0.101057,-0.739130,826579,25.224449,252.244492,14.880610
32,神奈川県,7392,66605,944,2021-06-26,231,0,1305.0,0.045249,0.045773,0.276243,9209442,72.322514,723.225142,14.170240
5,千葉県,6336,39811,704,2021-06-26,108,1,756.0,-0.142857,0.021467,-0.092437,6319772,62.994361,629.943612,11.962457
44,高知県,20592,1788,24,2021-06-26,11,0,73.0,0.222222,0.150624,0.100000,709230,25.210439,252.104395,10.292853
7,埼玉県,5808,46097,830,2021-06-26,96,0,641.0,-0.040000,0.066528,0.215190,7390054,62.377081,623.770814,8.673820
10,奈良県,15312,8133,129,2021-06-26,20,0,115.0,0.176471,0.443878,1.000000,1353837,60.073702,600.737016,8.494376
27,栃木県,4752,6939,81,2021-06-26,37,0,163.0,0.370370,0.196981,1.642857,1965516,35.303707,353.037065,8.292988


## 直近１週間の新規感染者 増加速度

## 増加速度 統計情報

In [16]:
all_jp["直近1週間の新規感染者数前日比平均"].describe()

count    32.000000
mean      0.331630
std       0.414069
min      -0.142011
25%       0.093524
50%       0.186550
75%       0.372648
max       1.612670
Name: 直近1週間の新規感染者数前日比平均, dtype: float64

### WORST 10(増加速度の早い順)

In [17]:
all_jp.sort_values("直近1週間の新規感染者数前日比平均",ascending=False).head(10)

,都道府県名,都道府県コード,各地の感染者数合計,各地の死者数合計,日付,各地の感染者数_1日ごとの発表数,各地の死者数_1日ごとの発表数,直近1週間の感染者数,新規感染者数前日比,直近1週間の新規感染者数前日比平均,新規感染者数先週比,人口,１万人当たりの感染者数,10万人当たりの感染者数,直近1週間の10万人当たりの感染者数
1,京都府,13728,16539,241,2021-06-26,20,0,90.0,1.000000,1.612670,0.250000,2545899,64.963300,649.632998,3.535097
29,滋賀県,13200,5537,90,2021-06-26,2,0,52.0,-0.818182,1.483317,-0.818182,1420948,38.966943,389.669432,3.659529
36,秋田県,2640,825,20,2021-06-26,21,0,60.0,0.750000,0.986848,20.000000,985416,8.372099,83.720987,6.088799
13,富山県,8448,2036,37,2021-06-26,2,0,30.0,-0.600000,0.961905,-0.666667,1055999,19.280321,192.803213,2.840912
0,三重県,12672,5243,110,2021-06-26,5,0,62.0,-0.772727,0.746204,0.250000,1813859,28.905224,289.052236,3.418127
30,熊本県,22704,6472,115,2021-06-26,2,1,40.0,-0.714286,0.623222,1.000000,1769880,36.567451,365.674509,2.260040
10,奈良県,15312,8133,129,2021-06-26,20,0,115.0,0.176471,0.443878,1.000000,1353837,60.073702,600.737016,8.494376
46,鹿児島県,24288,3653,38,2021-06-26,6,0,21.0,2.000000,0.380952,2.000000,1630146,22.409036,224.090358,1.288228
38,茨城県,4224,10440,162,2021-06-26,31,0,186.0,-0.446429,0.369879,-0.060606,2921436,35.735850,357.358505,6.366732
14,山口県,18480,3130,77,2021-06-26,5,0,29.0,-0.166667,0.327891,-0.166667,1369882,22.848683,228.486833,2.116971


### BEST 10(増加速度の遅い順／減少速度の速い順)

In [18]:
all_jp.sort_values("直近1週間の新規感染者数前日比平均",ascending=True).head(10)

,都道府県名,都道府県コード,各地の感染者数合計,各地の死者数合計,日付,各地の感染者数_1日ごとの発表数,各地の死者数_1日ごとの発表数,直近1週間の感染者数,新規感染者数前日比,直近1週間の新規感染者数前日比平均,新規感染者数先週比,人口,１万人当たりの感染者数,10万人当たりの感染者数,直近1週間の10万人当たりの感染者数
21,広島県,17952,11468,174,2021-06-26,5,0,99.0,-0.583333,-0.142011,-0.772727,2826858,40.568009,405.680087,3.502121
16,山梨県,10032,2085,21,2021-06-26,12,0,123.0,-0.142857,-0.101057,-0.739130,826579,25.224449,252.244492,14.880610
4,北海道,528,41216,1382,2021-06-26,47,5,273.0,0.424242,0.017342,-0.356164,5267762,78.241956,782.419555,5.182466
5,千葉県,6336,39811,704,2021-06-26,108,1,756.0,-0.142857,0.021467,-0.092437,6319772,62.994361,629.943612,11.962457
32,神奈川県,7392,66605,944,2021-06-26,231,0,1305.0,0.045249,0.045773,0.276243,9209442,72.322514,723.225142,14.170240
7,埼玉県,5808,46097,830,2021-06-26,96,0,641.0,-0.040000,0.066528,0.215190,7390054,62.377081,623.770814,8.673820
34,福岡県,21120,35414,519,2021-06-26,40,1,234.0,0.081081,0.068368,-0.148936,5129841,69.035278,690.352781,4.561545
24,愛知県,12144,50930,955,2021-06-26,46,5,501.0,-0.303030,0.081240,-0.483146,7575530,67.229620,672.296196,6.613399
31,石川県,8976,3927,115,2021-06-26,0,0,16.0,-1.000000,0.097619,-1.000000,1139612,34.459097,344.590966,1.403987
26,東京都,6864,172041,2222,2021-06-26,534,4,3332.0,-0.049822,0.102846,0.376289,13834925,124.352680,1243.526799,24.083976


## グラフ化

In [19]:
import matplotlib.pyplot as plt
%matplotlib notebook
#%matplotlib inline

In [20]:
all_jp_desc = all_jp.sort_values(["各地の感染者数合計"],ascending=False)
all_jp_desc_10k = all_jp.sort_values(["１万人当たりの感染者数"],ascending=False)
all_jp_desc_100k = all_jp.sort_values(["直近1週間の10万人当たりの感染者数"],ascending=False)

In [21]:
fig = plt.figure(figsize=(9.5,5))
plt.subplot(1,2,1) # (rows, columns, panel number)
plt.title("各地の新規感染者数累計")
plt.yticks(fontsize=8)
plt.grid(axis="x")
chart_data = all_jp.sort_values(["各地の感染者数合計"])
plt.barh(chart_data["都道府県名"],chart_data["各地の感染者数合計"],align="center")

plt.subplot(1,2,2) # (rows, columns, panel number)
plt.title("1万人当たりの新規感染者数累計")
plt.yticks(fontsize=8)
plt.grid(axis="x")
chart_data = all_jp.sort_values(["１万人当たりの感染者数"])
plt.barh(chart_data["都道府県名"], chart_data["１万人当たりの感染者数"])

fig.text(0,0,"※新型コロナ関連の情報提供:NHK ※人口の情報提供:e-Stat",fontsize=8)
fig.text(0,0.98,"※{}時点".format(latest_date),fontsize=9,va="top")

fig.tight_layout()

<IPython.core.display.Javascript object>

In [22]:
fig = plt.figure(figsize=(9.5,10))

### 最新の各地の新規感染者数
plt.subplot(2,2,1) # (rows, columns, panel number)
plt.title("{}の各地の新規感染者数".format(latest_date))
chart_data = all_jp.sort_values(["各地の感染者数_1日ごとの発表数"])
plt.barh(chart_data["都道府県名"], chart_data["各地の感染者数_1日ごとの発表数"])
plt.yticks(fontsize=8)
plt.grid(axis="x")

### 最新の各地の死者数
plt.subplot(2,2,2) # (rows, columns, panel number)
plt.title("{}の各地の新規死者数".format(latest_date))
chart_data = all_jp.sort_values(["各地の死者数_1日ごとの発表数"])
plt.barh(chart_data["都道府県名"], chart_data["各地の死者数_1日ごとの発表数"])
plt.yticks(fontsize=8)
plt.grid(axis="x")



### 直近1週間の10万人当たりの新規感染者数
plt.subplot(2,2,3) # (rows, columns, panel number)
plt.title("直近1週間の10万人当たりの新規感染者数")
chart_data = all_jp.sort_values(["直近1週間の10万人当たりの感染者数"])
plt.barh(chart_data["都道府県名"], chart_data["直近1週間の10万人当たりの感染者数"])
plt.plot([15,15],[0,47],"--",lw=0.5,color="black")
plt.text(15,0,"Stage3",ha="center",va="top",fontsize=8)
plt.plot([25,25],[0,47],"--",lw=0.5,color="black")
plt.text(25,0,"Stage4",ha="center",va="top",fontsize=8)
plt.yticks(fontsize=8)
plt.grid(axis="x")

### 新規感染者数の増加速度（新規感染者数前週同曜日比）
plt.subplot(2,2,4) # (rows, columns, panel number)
plt.title("新規感染者数前週同曜日比")
chart_data = all_jp.sort_values("新規感染者数先週比",na_position="first")
plt.barh(chart_data["都道府県名"], chart_data["新規感染者数先週比"])
plt.yticks(fontsize=8)
plt.grid(axis="x")

fig.text(0,0,"※新型コロナ関連の情報提供:NHK ※人口の情報提供:e-Stat",fontsize=8)
fig.text(0,0.98,"※{}時点".format(latest_date),fontsize=9,va="top")

fig.tight_layout()

<IPython.core.display.Javascript object>

In [23]:
fig.savefig("out/covid-domestic.png")

## 度数分布表

In [24]:
import numpy as np
data = np.array(all_jp["各地の感染者数合計"])
 
# ヒストグラム
hist, bin_edges = np.histogram(data, bins=8)

print(hist)
print(bin_edges)

[38  4  2  1  1  0  0  1]
[   466.     21912.875  43359.75   64806.625  86253.5   107700.375
 129147.25  150594.125 172041.   ]


## ヒストグラム

In [25]:
fig = plt.figure(figsize=(10,3))
plt.hist(all_jp_desc["各地の感染者数合計"],bins=[0,20000,40000,80000,100000,120000,140000,160000])

<IPython.core.display.Javascript object>

(array([37.,  3.,  5.,  0.,  1.,  0.,  0.]),
 array([     0,  20000,  40000,  80000, 100000, 120000, 140000, 160000]),
 <a list of 7 Patch objects>)

In [26]:
fig = plt.figure(figsize=(10,3))
plt.hist(all_jp_desc["10万人当たりの感染者数"],bins=[0,200,400,600,800,1000,1200])


<IPython.core.display.Javascript object>

(array([ 9., 23.,  3.,  9.,  0.,  1.]),
 array([   0,  200,  400,  600,  800, 1000, 1200]),
 <a list of 6 Patch objects>)

In [27]:
pref[pref["都道府県名"]=="富山県"].tail(10)


,日付,都道府県コード,都道府県名,各地の感染者数_1日ごとの発表数,各地の感染者数_累計,各地の死者数_1日ごとの発表数,各地の死者数_累計,直近1週間の感染者数,新規感染者数前日比,直近1週間の新規感染者数前日比平均,新規感染者数先週比
8438,2021-06-17,16,富山県,11,1985,0,37,37.0,1.200000,0.628571,1.200000
8439,2021-06-18,16,富山県,15,2000,0,37,51.0,0.363636,0.794805,14.000000
8440,2021-06-19,16,富山県,6,2006,0,37,52.0,-0.600000,0.137662,0.200000
8441,2021-06-20,16,富山県,3,2009,0,37,50.0,-0.500000,0.066234,-0.400000
8442,2021-06-21,16,富山県,8,2017,0,37,53.0,1.666667,0.304329,0.600000
8443,2021-06-22,16,富山県,1,2018,0,37,49.0,-0.875000,0.179329,-0.800000
8444,2021-06-23,16,富山県,8,2026,0,37,52.0,7.000000,1.179329,0.600000
8445,2021-06-24,16,富山県,3,2029,0,37,44.0,-0.625000,0.918615,-0.727273
8446,2021-06-25,16,富山県,5,2034,0,37,34.0,0.666667,0.961905,-0.666667
8447,2021-06-26,16,富山県,2,2036,0,37,30.0,-0.600000,0.961905,-0.666667
